<h1>SQL Workshop Week 3 - Solutions</h1>
<p>This notebook contains exploratory queries used during the workshop, together with answers to the 15 SQL questions</p>

# Movie Ratings

In [4]:
import pandas as pd
import sqlalchemy

You've started a new movie-rating website, and you've been collecting data on reviewers' ratings of various movies. There's not much data yet, but you can still try out some interesting queries. Here's the schema: 

Movie ( mID, title, year, director ) 
English: There is a movie with ID number mID, a title, a release year, and a director. 

Reviewer ( rID, name ) 
English: The reviewer with ID number rID has a certain name. 

Rating ( rID, mID, stars, ratingDate ) 
English: The reviewer rID gave the movie mID a number of stars rating (1-5) on a certain ratingDate. 

## Connect to the database

Make sure you apply the `Run PostgreSQL` environment on the server

In [5]:
engine = sqlalchemy.create_engine(
    "postgresql+psycopg2://", 
    connect_args={"database": "movies", "user": "faculty","host": "/var/run/postgresql"}
)
con = engine.connect()
con.execute("SET schema 'public'")

## Load the data in the db
If you want to reset the database, simply rerun this cell

In [6]:
from sqlalchemy.sql import text
con.execute(open("movie-ratings.sql").read())
con.execute(open("social.sql").read())

In [8]:
q = """
SELECT * 
FROM Rating 
limit 3
"""
df = pd.read_sql(q, con)
df

,rid,mid,stars,ratingdate
0,201,101,2,2011-01-22
1,201,101,4,2011-01-27
2,202,106,4,None


Find the titles of all movies directed by Steven Spielberg. 


In [8]:
q = """
select title
from Movie
where director='Steven Spielberg';
"""
df = pd.read_sql(q, con)
df

,title
0,E.T.
1,Raiders of the Lost Ark


Find all years that have a movie that received a rating of 4 or 5, and sort them in increasing order. 

In [9]:
q = """
select distinct year
from Movie, Rating
where Rating.mID = Movie.mID and stars >= 4
order by Year;
"""
df = pd.read_sql(q, con)
df

,year
0,1937
1,1939
2,1981
3,2009


Find the titles of all movies that have no ratings. 


In [10]:
 q = """
select title
from Movie
where mID not in (select mID 
                  from Rating)
"""
df = pd.read_sql(q, con)
df

,title
0,Star Wars
1,Titanic


Some reviewers didn't provide a date with their rating. Find the names of all reviewers who have ratings with a NULL value for the date. 


In [11]:
 q = """
select distinct name
from Reviewer, Rating
where Reviewer.rID = Rating.rID and ratingDate is NULL;
"""
df = pd.read_sql(q, con)
df

,name
0,Daniel Lewis
1,Chris Jackson


Write a query to return the ratings data in a more readable format: reviewer name, movie title, stars, and ratingDate. Also, sort the data, first by reviewer name, then by movie title, and lastly by number of stars. 
 

In [12]:
 q = """
select name, title, stars, ratingDate
from Movie, Rating, Reviewer
where Movie.mID = Rating.mID and Reviewer.rID = Rating.rID
order by name, title, stars;
"""
df = pd.read_sql(q, con)
df

,name,title,stars,ratingdate
0,Ashley White,E.T.,3,2011-01-02
1,Brittany Harris,Raiders of the Lost Ark,2,2011-01-30
2,Brittany Harris,Raiders of the Lost Ark,4,2011-01-12
3,Brittany Harris,The Sound of Music,2,2011-01-20
4,Chris Jackson,E.T.,2,2011-01-22
5,Chris Jackson,Raiders of the Lost Ark,4,None
6,Chris Jackson,The Sound of Music,3,2011-01-27
7,Daniel Lewis,Snow White,4,None
8,Elizabeth Thomas,Avatar,3,2011-01-15
9,Elizabeth Thomas,Snow White,5,2011-01-19


For all cases where the same reviewer rated the same movie twice and gave it a higher rating the second time, return the reviewer's name and the title of the movie. 

In [13]:
 q = """
select name, title
from Movie, Reviewer, (select R1.rID, R1.mID
  from Rating R1, Rating R2
  where R1.rID = R2.rID 
  and R1.mID = R2.mID
  and R1.stars < R2.stars
  and R1.ratingDate < R2.ratingDate) C
where Movie.mID = C.mID
and Reviewer.rID = C.rID;
"""
df = pd.read_sql(q, con)
df

,name,title
0,Sarah Martinez,Gone with the Wind


For each movie that has at least one rating, find the highest number of stars that movie received. Return the movie title and number of stars. Sort by movie title. 

In [ ]:
 q = """
select title, stars
from Movie, ( select Movie.mID, stars
              from Movie, Rating
              where Movie.mID = Rating.mID
              except
              select R1.mID, R1.stars
              from Rating R1, Rating R2
              where R1.mID = R2.mID
              and R1.stars < R2.stars) Stars
where Movie.mID = Stars.mID
order by title;
"""
df = pd.read_sql(q, con)
df

For each movie, return the title and the 'rating spread', that is, the difference between highest and lowest ratings given to that movie. Sort by rating spread from highest to lowest, then by movie title. 

In [14]:
 q = """
select title, spread
from Movie, (
  select mID, max(stars) - min(stars) as spread
  from Rating
  group by mID
) RatingSpread
where Movie.mID = RatingSpread.mID
order by spread DESC, title;
"""
df = pd.read_sql(q, con)
df

,title,spread
0,Avatar,2
1,Gone with the Wind,2
2,Raiders of the Lost Ark,2
3,E.T.,1
4,Snow White,1
5,The Sound of Music,1


Find the difference between the average rating of movies released before 1980 and the average rating of movies released after 1980. (Make sure to calculate the average rating for each movie, then the average of those averages for movies before 1980 and movies after. Don't just calculate the overall average rating before and after 1980.) 

In [15]:
 q = """
select avg(before_80.group_avg) - avg(post_80.group_avg) as difference
from (
  select Rating.mID, avg(stars) as group_avg
  from Rating, Movie
  where Rating.mID = Movie.mID
  and year <= 1980
  group by Rating.mID
) as before_80,
(
  select Rating.mID, avg(stars) as group_avg
  from Rating, Movie
  where Rating.mID = Movie.mID
  and year > 1980
  group by Rating.mID
) as post_80
"""
df = pd.read_sql(q, con)
df

,difference
0,0.055556


# The Social Network

Students at your hometown high school have decided to organize their social network using databases. So far, they have collected information about sixteen students in four grades, 9-12. Here's the schema: 

Highschooler ( ID, name, grade ) 
English: There is a high school student with unique ID and a given first name in a certain grade. 

Friend ( ID1, ID2 ) 
English: The student with ID1 is friends with the student with ID2. Friendship is mutual, so if (123, 456) is in the Friend table, so is (456, 123). 

Likes ( ID1, ID2 ) 
English: The student with ID1 likes the student with ID2. Liking someone is not necessarily mutual, so if (123, 456) is in the Likes table, there is no guarantee that (456, 123) is also present. 


In [16]:
df = pd.read_sql("SELECT * from Highschooler limit 3", con)
df

,id,name,grade
0,1510,Jordan,9
1,1689,Gabriel,9
2,1381,Tiffany,9


Find the names of all students who are friends with someone named Gabriel. 


In [17]:
 q = """
select H2.name
from Highschooler H1, Highschooler H2, Friend
where H1.ID = Friend.ID1
and H1.name = 'Gabriel'
and H2.ID = Friend.ID2;
"""
df = pd.read_sql(q, con)
df

,name
0,Jordan
1,Cassandra
2,Andrew
3,Alexis
4,Jessica


For every student who likes someone 2 or more grades younger than themselves, return that student's name and grade, and the name and grade of the student they like. 

In [18]:
 q = """
select H1.name, H1.grade, H2.name, H2.grade
from Likes, Highschooler H1, Highschooler H2
where Likes.ID1 = H1.ID
and Likes.ID2 = H2.ID
and H1.grade >= (H2.grade + 2)
"""
df = pd.read_sql(q, con)
df

,name,grade,name,grade
0,John,12,Haley,10


For every pair of students who both like each other, return the name and grade of both students. Include each pair only once, with the two names in alphabetical order. 

In [19]:
 q = """
select H1.name, H1.grade, H2.name, H2.grade
from Highschooler H1, Highschooler H2, (
  select L1.ID1, L1.ID2
  from Likes L1, Likes L2
  where L1.ID2 = L2.ID1
  and L1.ID1 = L2.ID2
) as Pair
where H1.ID = Pair.ID1
and H2.ID = Pair.ID2
and H1.name < H2.name
"""
df = pd.read_sql(q, con)
df

,name,grade,name,grade
0,Cassandra,9,Gabriel,9
1,Jessica,11,Kyle,12


Find all students who do not appear in the Likes table (as a student who likes or is liked) and return their names and grades. Sort by grade, then by name within each grade. 

In [20]:
 q = """
select name, grade
from Highschooler
where ID not in (
  select ID1 from Likes
  union
  select ID2 from Likes
)
order by grade, name
"""
df = pd.read_sql(q, con)
df

,name,grade
0,Jordan,9
1,Tiffany,9
2,Logan,12


For every situation where student A likes student B, but we have no information about whom B likes (that is, B does not appear as an ID1 in the Likes table), return A and B's names and grades. 

In [21]:
 q = """
select H1.name, H1.grade, H2.name, H2.grade, H3.name, H3.grade
from Likes L1, Likes L2, Highschooler H1, Highschooler H2, Highschooler H3
where L1.ID2 = L2.ID1
and L2.ID2 <> L1.ID1
and L1.ID1 = H1.ID and L1.ID2 = H2.ID and L2.ID2 = H3.ID
"""
df = pd.read_sql(q, con)
df

,name,grade,name,grade,name,grade
0,Gabriel,11,Alexis,11,Kris,10
1,Andrew,10,Cassandra,9,Gabriel,9


Find names and grades of students who only have friends in the same grade. Return the result sorted by grade, then by name within each grade. 

In [22]:
 q = """
select name, grade
from Highschooler, (
  select ID1 from Friend
  except
  -- students have friends in same grade
  select distinct Friend.ID1
  from Friend, Highschooler H1, Highschooler H2
  where Friend.ID1 = H1.ID and Friend.ID2 = H2.ID
  and H1.grade != H2.grade
) as Sample
where Highschooler.ID = Sample.ID1
order by grade, name
"""
df = pd.read_sql(q, con)
df

,name,grade
0,Jordan,9
1,Brittany,10
2,Haley,10
3,Kris,10
4,Gabriel,11
5,John,12
6,Logan,12


For each student A who likes a student B where the two are not friends, find if they have a friend C in common (who can introduce them!). For all such trios, return the name and grade of A, B, and C. 

In [23]:
 q = """
select H1.name, H1.grade, H2.name, H2.grade, H3.name, H3.grade
from Highschooler H1, Highschooler H2, Highschooler H3, Friend F1, Friend F2, (
  select * from Likes
  except
  -- A likes B and A/B are friends
  select Likes.ID1, Likes.ID2
  from Likes, Friend
  where Friend.ID1 = Likes.ID1 and Friend.ID2 = Likes.ID2
) as LikeNotFriend
where F1.ID1 = LikeNotFriend.ID1
and F2.ID1 = LikeNotFriend.ID2
-- has a shared friend
and F1.ID2 = F2.ID2
and H1.ID = LikeNotFriend.ID1
and H2.ID = LikeNotFriend.ID2
and H3.ID = F2.ID2
"""
df = pd.read_sql(q, con)
df

,name,grade,name,grade,name,grade
0,Austin,11,Jordan,12,Andrew,10
1,Austin,11,Jordan,12,Kyle,12
2,Andrew,10,Cassandra,9,Gabriel,9


Find the difference between the number of students in the school and the number of different first names. 


In [24]:
 q = """
select count(ID) - count(distinct name) as difference
from Highschooler
"""
df = pd.read_sql(q, con)
df

,difference
0,2


Find the name and grade of all students who are liked by more than one other student. 


In [25]:
q = """
select name, grade
from Highschooler, (
  select count(ID1) as count, ID2
  from Likes
  group by ID2
) as LikeCount
where Highschooler.ID = LikeCount.ID2
and count > 1
"""
df = pd.read_sql(q, con)
df

,name,grade
0,Cassandra,9
1,Kris,10


## Modification

Add the reviewer Roger Ebert to your database, with an rID of 209. 


In [26]:
q = "insert into Reviewer(rID, name) values (209, 'Roger Ebert')"
con.execute(q)

Insert 5-star ratings by James Cameron for all movies in the database. Leave the review date as NULL. 


In [27]:
q = """
insert into Rating
  select Rating.rID, Movie.mID, 5 as stars, null as ratingDate
  from Rating, Movie, Reviewer
  where Rating.rID = Reviewer.rID
  and Reviewer.name = 'James Cameron';
"""
con.execute(q)

For all movies that have an average rating of 4 stars or higher, add 25 to the release year. (Update the existing tuples; don't insert new tuples.) 

In [28]:
q = """
update Movie
set year = year + 25
where mID in (
  select Movie.mId
  from Movie, Rating
  where Movie.mID = Rating.mID
  group by Movie.mID
  having avg(stars) >= 4
)
"""
con.execute(q)

Remove all ratings where the movie's year is before 1970 or after 2000, and the rating is fewer than 4 stars. 

In [29]:
q = """
delete from Rating
where mID in (
  select distinct Rating.mID
  from Movie, Rating
  where Movie.mID = Rating.mID
  and (Movie.year > 2000 or Movie.year < 1970)
)
and stars < 4
"""
con.execute(q)

It's time for the seniors to graduate. Remove all 12th graders from Highschooler. 


In [30]:
q = """
delete from Highschooler
where grade = 12
"""
con.execute(q)

If two students A and B are friends, and A likes B but not vice-versa, remove the Likes tuple. 


In [31]:
q = """
delete from Likes l
where l.ID1 in (
  select a.ID1 from (
    select L1.ID1, L1.ID2
    from Friend, Likes L1
    where Friend.ID1 = L1.ID1
    and Friend.ID2 = L1.ID2
    except
    select L1.ID1, L1.ID2
    from Likes L1, Likes L2
    where L1.ID1 = L2.ID2
    and L1.ID2 = L2.ID1
  ) a
)
"""
con.execute(q)

For all cases where A is friends with B, and B is friends with C, add a new friendship for the pair A and C. Do not add duplicate friendships, friendships that already exist, or friendships with oneself. (This one is a bit challenging; congratulations if you get it right.) 

In [32]:
q = """
insert into Friend
  select F1.ID1, F2.ID2
  from Friend F1, Friend F2
  where F1.ID2 = F2.ID1
  -- friends with oneself
  and F1.ID1 != F2.ID2
  -- already exist friendship
  except 
  select * from Friend
"""
con.execute(q)

<h1>SQL Workshop - Week 3 - Solutions </h1>

<h4>Exercise 1 </h4>
<p> You now have all of the data from the  table. But maybe you only want certain columns from thattable. To limit this, replace the ​*​ with the column names that you want. </p>


In [9]:
q = """
SELECT name, grade 
FROM Highschooler 
"""
df = pd.read_sql(q, con)
df

,name,grade
0,Jordan,9
1,Gabriel,9
2,Tiffany,9
3,Cassandra,9
4,Haley,10
5,Andrew,10
6,Kris,10
7,Brittany,10
8,Alexis,11
9,Austin,11


<h4>Exercise 2 </h4>
<p> Let’s start putting SQL to the test by limiting what we get back. We do this using by adding a WHERE command, after which we can add some conditions</p>

In [10]:
q = """
SELECT name, grade 
FROM Highschooler WHERE grade=12
"""
df = pd.read_sql(q, con)
df

,name,grade
0,Jordan,12
1,John,12
2,Kyle,12
3,Logan,12


<h4>Exercise 3 </h4>
<p>Write a statement that returns all the movies released in 2000 </p>

In [12]:
q = """
SELECT * 
FROM Movie
"""
df = pd.read_sql(q, con)
df

,mid,title,year,director
0,101,Gone with the Wind,1939,Victor Fleming
1,102,Star Wars,1977,George Lucas
2,103,The Sound of Music,1965,Robert Wise
3,104,E.T.,1982,Steven Spielberg
4,105,Titanic,1997,James Cameron
5,106,Snow White,1937,None
6,107,Avatar,2009,James Cameron
7,108,Raiders of the Lost Ark,1981,Steven Spielberg


In [14]:
q = """
SELECT title 
FROM Movie
WHERE year=2000
"""
df = pd.read_sql(q, con)
df

,title


In our database, there is no record of movies release in 2000.

<h4>Exercise 4 </h4>
<p>Alter your last statement so you only select movies released between 2000 and 2005 </p>

In [16]:
q = """
SELECT title 
FROM Movie
WHERE year>=2000 AND year<=2005
"""
df = pd.read_sql(q, con)
df

,title


What if we only want movies produced before 2000?

In [18]:
q = """
SELECT title,year 
FROM Movie
WHERE year < 2000
"""
df = pd.read_sql(q, con)
df

,title,year
0,Gone with the Wind,1939
1,Star Wars,1977
2,The Sound of Music,1965
3,E.T.,1982
4,Titanic,1997
5,Snow White,1937
6,Raiders of the Lost Ark,1981


<h4>Exercise 5 </h4>
<p> Alter the statement again to limit your movies to just those produced by Steven Spielberg. </p>

In [19]:
q = """
SELECT title,year 
FROM Movie
WHERE year < 2000 AND director=\'Steven Spielberg\'
"""
df = pd.read_sql(q, con)
df

,title,year
0,E.T.,1982
1,Raiders of the Lost Ark,1981


<h4>Exercise 6 </h4>
<p> Count the total number of movies with a lower rating than 3</p>
<p>Here, things can get a bit tricky. We don't have the movies and their ratings in one table, hence we will have to join the Movie and Rating tables</p>

In [20]:
q = """
SELECT * FROM Rating
"""
df = pd.read_sql(q, con)
df

,rid,mid,stars,ratingdate
0,201,101,2,2011-01-22
1,201,101,4,2011-01-27
2,202,106,4,None
3,203,103,2,2011-01-20
4,203,108,4,2011-01-12
5,203,108,2,2011-01-30
6,204,101,3,2011-01-09
7,205,103,3,2011-01-27
8,205,104,2,2011-01-22
9,205,108,4,None


In [21]:
q = """
SELECT * 
FROM Movie
"""
df = pd.read_sql(q, con)
df

,mid,title,year,director
0,101,Gone with the Wind,1939,Victor Fleming
1,102,Star Wars,1977,George Lucas
2,103,The Sound of Music,1965,Robert Wise
3,104,E.T.,1982,Steven Spielberg
4,105,Titanic,1997,James Cameron
5,106,Snow White,1937,None
6,107,Avatar,2009,James Cameron
7,108,Raiders of the Lost Ark,1981,Steven Spielberg


In [22]:
q = """
SELECT COUNT(*) FROM (
SELECT * FROM Movie
LEFT JOIN Rating ON Movie.mid = Rating.mid
) joined_table WHERE joined_table.stars < 3
"""
df = pd.read_sql(q, con)
df

,count
0,4


<h4>Exercise 7 </h4>
<p> Let’s find out how many movies produced by Steven Spielberg have a rating lower than 3 (We’re guessing it’s a very smallnumber) </p>

In [23]:
q = """
SELECT COUNT(*) FROM (
SELECT * FROM Movie
LEFT JOIN Rating ON Movie.mid = Rating.mid
) joined_table WHERE joined_table.stars < 3 AND joined_table.director=\'Steven Spielberg\'
"""
df = pd.read_sql(q, con)
df

,count
0,2


In [24]:
#Let's now check which movies are produced by Steven Spielberg and have a lower rating than 3. 
q = """
SELECT *FROM (
SELECT * FROM Movie
LEFT JOIN Rating ON Movie.mid = Rating.mid
) joined_table WHERE joined_table.stars < 3 AND joined_table.director=\'Steven Spielberg\'
"""
df = pd.read_sql(q, con)
df

,mid,title,year,director,rid,mid,stars,ratingdate
0,108,Raiders of the Lost Ark,1981,Steven Spielberg,203,108,2,2011-01-30
1,104,E.T.,1982,Steven Spielberg,205,104,2,2011-01-22


It seems like E.T. did pretty poorly

<h4>Exercise 8 </h4>
<p> Use the ​SELECT DISTINCT​ syntax to count the number of distinct directors in our movies rating database</p>

In [68]:
q = """
SELECT DISTINCT director FROM Movie
"""
df = pd.read_sql(q, con)
df

,director
0,None
1,James Cameron
2,Steven Spielberg
3,Robert Wise
4,George Lucas
5,Victor Fleming


In [71]:
q = """
SELECT COUNT(DISTINCT director) FROM Movie
"""
df = pd.read_sql(q, con)
df

,count
0,5


<h4>Exercise 9 </h4>
<p>Try finding out the mean rating for movies produced by Steven Spielberg </p>

In [27]:
q = """
SELECT AVG(stars) FROM (
SELECT * FROM Movie
LEFT JOIN Rating ON Movie.mid = Rating.mid
) joined_table WHERE joined_table.stars < 3 AND joined_table.director=\'Steven Spielberg\'
"""
df = pd.read_sql(q, con)
df

,avg
0,2.0


<h4>Exercise 10 </h4>
<p> Order students by the number of likes. We’d use our ​ORDER BY​ command followed by a field representing the total number of likes. ​Give it a try.</p>

In [33]:
q = """
SELECT * FROM Likes
"""
df = pd.read_sql(q, con)
df

,id1,id2
0,1689,1709
1,1709,1689
2,1782,1709
3,1911,1247
4,1247,1468
5,1641,1468
6,1316,1304
7,1501,1934
8,1934,1501
9,1025,1101


Let's count the number of likes for each student. In order to so, we will count the number of occurences of each id in id2.


In [39]:
#Please note we had to group by id2 in order to execute this query.
q = """
SELECT id2, COUNT(id2) FROM Likes
GROUP BY id2
"""
df = pd.read_sql(q, con)
df

,id2,count
0,1689,1
1,1501,1
2,1468,2
3,1247,1
4,1934,1
5,1304,1
6,1709,2
7,1101,1


In [35]:
q = """
SELECT * FROM Highschooler
"""
df = pd.read_sql(q, con)
df

,id,name,grade
0,1510,Jordan,9
1,1689,Gabriel,9
2,1381,Tiffany,9
3,1709,Cassandra,9
4,1101,Haley,10
5,1782,Andrew,10
6,1468,Kris,10
7,1641,Brittany,10
8,1247,Alexis,11
9,1316,Austin,11


In [45]:
#Let's join the table registering the number of likes for each student id to the Highschooler student to have a better understanding of who is more popular .
q = """
SELECT * FROM (

SELECT * FROM Highschooler
LEFT JOIN (SELECT id2, COUNT(id2) FROM Likes
GROUP BY id2) Popularity ON Highschooler.id=Popularity.id2
) joined_table ORDER BY joined_table.count
"""
df = pd.read_sql(q, con)
df

,id,name,grade,id2,count
0,1247,Alexis,11,1247.0,1.0
1,1101,Haley,10,1101.0,1.0
2,1689,Gabriel,9,1689.0,1.0
3,1501,Jessica,11,1501.0,1.0
4,1304,Jordan,12,1304.0,1.0
5,1934,Kyle,12,1934.0,1.0
6,1709,Cassandra,9,1709.0,2.0
7,1468,Kris,10,1468.0,2.0
8,1661,Logan,12,NaN,NaN
9,1381,Tiffany,9,NaN,NaN


We can see here that the order above is ascending, yet the null values prevent us from gaining an accurate insight. Clean the data and re-run the query
In order to do so, we will have to use a CASE statement. For more information about the CASE statement, read this article: https://www.datacamp.com/community/tutorials/case-statements-in-postgresql
<br/>
<br/>
 Also, have a look at the Postgres documentation for a CASE statement https://www.postgresql.org/docs/7.4/functions-conditional.html

In [55]:
q = """
SELECT joined_table.id,joined_table.name,joined_table.grade,CASE WHEN joined_table.id2 IS NULL THEN CAST(floor(random() * 10 + 1) AS INT) ELSE joined_table.id2 END
, CASE WHEN joined_table.count IS NULL THEN CAST(0 AS INT) ELSE CAST(joined_table.count AS INT) END  FROM (
SELECT * FROM Highschooler
LEFT JOIN (SELECT id2, COUNT(id2) FROM Likes
GROUP BY id2) Popularity ON Highschooler.id=Popularity.id2
) joined_table ORDER BY joined_table.count
"""

df = pd.read_sql(q, con)
df

,id,name,grade,id2,count
0,1247,Alexis,11,1247,1
1,1101,Haley,10,1101,1
2,1689,Gabriel,9,1689,1
3,1501,Jessica,11,1501,1
4,1304,Jordan,12,1304,1
5,1934,Kyle,12,1934,1
6,1709,Cassandra,9,1709,2
7,1468,Kris,10,1468,2
8,1661,Logan,12,2,0
9,1381,Tiffany,9,4,0


In [62]:
#Due to the PostgreSQL operation orders, we create a temporary table that we further order by count in order to get accurate results. 
q = """
CREATE TEMP TABLE joined_table AS (
SELECT joined_table.id,joined_table.name,joined_table.grade,CASE WHEN joined_table.id2 IS NULL THEN CAST(floor(random() * 10 + 1) AS INT) ELSE joined_table.id2 END
, CASE WHEN joined_table.count IS NULL THEN CAST(0 AS INT) ELSE CAST(joined_table.count AS INT) END  FROM (
SELECT * FROM Highschooler
LEFT JOIN (SELECT id2, COUNT(id2) FROM Likes
GROUP BY id2) Popularity ON Highschooler.id=Popularity.id2
) joined_table ORDER BY joined_table.count
)
"""
con.execute(q)

In [63]:
q="""SELECT * FROM joined_table
ORDER BY count
"""
df = pd.read_sql(q, con)
df

,id,name,grade,id2,count
0,1510,Jordan,9,3,0
1,1661,Logan,12,6,0
2,1381,Tiffany,9,1,0
3,1782,Andrew,10,4,0
4,1641,Brittany,10,3,0
5,1316,Austin,11,6,0
6,1911,Gabriel,11,6,0
7,1025,John,12,5,0
8,1247,Alexis,11,1247,1
9,1101,Haley,10,1101,1


In summary, we cleaned the data by giving a count of 0 likes to previous NULL count values and random integer values to the previous NULL id2 values.
We have then created a temporary table based on the query used for cleaning the data. 
Lastly, we queried the temporary table and ordered it by the count of likes.

<h4>Exercise 11 </h4>
<p> To reverse the popularity order of our high school pupils, we just need to add **DESC** after the ordered column name to find the least popular people.</p>

In [64]:
q="""SELECT * FROM joined_table
ORDER BY count DESC
"""
df = pd.read_sql(q, con)
df

,id,name,grade,id2,count
0,1468,Kris,10,1468,2
1,1709,Cassandra,9,1709,2
2,1501,Jessica,11,1501,1
3,1304,Jordan,12,1304,1
4,1934,Kyle,12,1934,1
5,1101,Haley,10,1101,1
6,1247,Alexis,11,1247,1
7,1689,Gabriel,9,1689,1
8,1510,Jordan,9,3,0
9,1661,Logan,12,6,0


In [66]:
#Since this was a temporary table, we will now delete the table from our database.
q="DROP TABLE joined_table"
con.execute(q)

<h4>Exercise 12 </h4>
<p>  Let’s say we want to know the total number of pupils grouped by grade. If we ​GROUP BY using ​grade​, we can shortcut to an answer quickly.</p>

In [20]:
q = """
SELECT grade,COUNT(*) FROM Highschooler
GROUP BY grade
ORDER BY grade ASC
"""
df = pd.read_sql(q, con)
df

,grade,count
0,9,4
1,10,4
2,11,4
3,12,4


It looks like there is an even distribution of grades.

<h4>Exercise 13 </h4>
<p>Insert a record in the Movie table and set the director’s name to Leonardo Dicaprio.</p>

In [21]:
q = """
SELECT * FROM Movie
"""
df = pd.read_sql(q, con)
df

,mid,title,year,director
0,101,Gone with the Wind,1939,Victor Fleming
1,102,Star Wars,1977,George Lucas
2,103,The Sound of Music,1965,Robert Wise
3,104,E.T.,1982,Steven Spielberg
4,105,Titanic,1997,James Cameron
5,106,Snow White,1937,None
6,107,Avatar,2009,James Cameron
7,108,Raiders of the Lost Ark,1981,Steven Spielberg


In [30]:
q = """
INSERT INTO Movie VALUES (109,'The Irishman',2019,'Leonardo DiCaprio')
"""
con.execute(q)

In [37]:
q = """
INSERT INTO Rating VALUES (209,109 ,4,'2020-02-05')
"""
con.execute(q)

In [31]:
q = """
SELECT * FROM Movie"""
df = pd.read_sql(q, con)
df

,mid,title,year,director
0,101,Gone with the Wind,1939,Victor Fleming
1,102,Star Wars,1977,George Lucas
2,103,The Sound of Music,1965,Robert Wise
3,104,E.T.,1982,Steven Spielberg
4,105,Titanic,1997,James Cameron
5,106,Snow White,1937,None
6,107,Avatar,2009,James Cameron
7,108,Raiders of the Lost Ark,1981,Steven Spielberg
8,109,The Irishman,2019,Leonardo DiCaprio


In [38]:
q = """
SELECT * FROM Rating"""
df = pd.read_sql(q, con)
df

,rid,mid,stars,ratingdate
0,201,101,2,2011-01-22
1,201,101,4,2011-01-27
2,202,106,4,None
3,203,103,2,2011-01-20
4,203,108,4,2011-01-12
5,203,108,2,2011-01-30
6,204,101,3,2011-01-09
7,205,103,3,2011-01-27
8,205,104,2,2011-01-22
9,205,108,4,None


<h4>Exercise 14 </h4>
<p> Change the rating of your made up movie to 5, and change the director name to your name .</p>

In [42]:
q = """
UPDATE Rating
SET  stars=5
WHERE mid=109 AND rid=209; 
"""
con.execute(q)

In [39]:
q = """
UPDATE Movie
SET  director='David Alderton'
WHERE mid=109
"""
con.execute(q)

In [43]:
q = """
SELECT * FROM Rating"""
df = pd.read_sql(q, con)
df

,rid,mid,stars,ratingdate
0,201,101,2,2011-01-22
1,201,101,4,2011-01-27
2,202,106,4,None
3,203,103,2,2011-01-20
4,203,108,4,2011-01-12
5,203,108,2,2011-01-30
6,204,101,3,2011-01-09
7,205,103,3,2011-01-27
8,205,104,2,2011-01-22
9,205,108,4,None


In [41]:
q = """
SELECT * FROM Movie"""
df = pd.read_sql(q, con)
df

,mid,title,year,director
0,101,Gone with the Wind,1939,Victor Fleming
1,102,Star Wars,1977,George Lucas
2,103,The Sound of Music,1965,Robert Wise
3,104,E.T.,1982,Steven Spielberg
4,105,Titanic,1997,James Cameron
5,106,Snow White,1937,None
6,107,Avatar,2009,James Cameron
7,108,Raiders of the Lost Ark,1981,Steven Spielberg
8,109,The Irishman,2019,David Alderton


<h4>Exercise 15 </h4>
<p>Create a DELETE FROM statement that deletes the row containing the made up movie you added to the Movie table earlier. </p>

In [44]:
q = """
DELETE FROM Movie
WHERE mid=109
"""
con.execute(q)

In [45]:
q = """
SELECT * FROM Movie"""
df = pd.read_sql(q, con)
df

,mid,title,year,director
0,101,Gone with the Wind,1939,Victor Fleming
1,102,Star Wars,1977,George Lucas
2,103,The Sound of Music,1965,Robert Wise
3,104,E.T.,1982,Steven Spielberg
4,105,Titanic,1997,James Cameron
5,106,Snow White,1937,None
6,107,Avatar,2009,James Cameron
7,108,Raiders of the Lost Ark,1981,Steven Spielberg


In [46]:
q = """
DELETE FROM Rating
WHERE mid=109
"""
con.execute(q)

In [47]:
q = """
SELECT * FROM Rating"""
df = pd.read_sql(q, con)
df

,rid,mid,stars,ratingdate
0,201,101,2,2011-01-22
1,201,101,4,2011-01-27
2,202,106,4,None
3,203,103,2,2011-01-20
4,203,108,4,2011-01-12
5,203,108,2,2011-01-30
6,204,101,3,2011-01-09
7,205,103,3,2011-01-27
8,205,104,2,2011-01-22
9,205,108,4,None
